In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
#from ed import * per importare ed senza bisogno di scrivere alcun prefisso
import ed

## Ex 7.1

a) As in Exercise 6.2, generate the ground state of the transverse field Ising model with
open boundary conditions for L = 14, g = 1.5, J ≡ 1. Make sure it is normalized to
⟨ψ|ψ⟩ = 1.

In [3]:
L = 14
g = 1.5
J = 1.
sx_list = ed.gen_sx_list(L)
sz_list = ed.gen_sz_list(L)
H = ed.gen_hamiltonian(sx_list, sz_list, g, J)
E, vecs = scipy.sparse.linalg.eigsh(H, which='SA')
psi0 = vecs[:, 0]
print(psi0)
np.linalg.norm(psi0)

[ 8.14694335e-01 -2.74452672e-15 -3.88226482e-15 ... -6.30369818e-18
 -8.00665169e-18  4.07427082e-06]


0.9999999999999983

b) Write a function compress(psi, L, chimax), which takes the state, the length of the
chain and the maximal desired bond dimension chimax as input and compresses the
state into MPS form using successive SVDs. It should return a list of L numpy
arrays, namely the M [n], each with 3 indices (αn, jn, αn+1).
Hint: Let us define the indices Rn = (jn, jn+1, . . . , jL), such that R1 ≡ i

In [4]:
def compress(psi, L, chimax):
    Ms = []
    psi = np.reshape(psi, (2, 2**(L-1)))
    n = 1
    lambda_n = [1]
    for i in range(1,L+1):
        psi = np.reshape(psi, (2*len(lambda_n),2**(L-i)))
        M_n, lambda_n, psitilde = np.linalg.svd(psi)

        keep = np.argsort ( lambda_n )[:: -1][: chimax ]
        M_n = M_n [: , keep ]
        lambda_n = lambda_n [ keep ]
        psitilde = psitilde [ keep , :]

        M_n = np.reshape(M_n, (n,2,len(lambda_n)))
        psi = lambda_n[:, np.newaxis] * psitilde[:, :]
        n = len(psitilde)

        Ms.append(M_n)
    
    return Ms
    



In [5]:
a = [1]
print(len(a))

1


c) What is the maximally necessary bond dimension for L = 14? Call compress() for
the ground state with χmax larger than that to get an exact MPS representation

The maximally necessary bond dimension is $2^{L/2} = 2^7$

In [6]:
chimax = 2**7
psiex = compress(psi0, 14, chimax)

In [7]:

entries_psiex = 0
for i in range(len(psiex)):
    Mi = np.array(psiex[i])
    a = np.shape(Mi)
    entries_psiex += a[0]*a[1]*a[2]


d) Call compress() again with χmax = 10 to get a compressed MPS. Compare
the number of floats stored in both MPS.
Hint: The number of elements in a numpy array M are given by M.size

In [8]:
psicompr = compress(psi0, 14, 10)

entries_psicompr = 0
for i in range(len(psicompr)):
    Mi = np.array(psicompr[i])
    a = np.shape(Mi)
    entries_psicompr += a[0]*a[1]*a[2]

In [9]:
print(entries_psicompr)
print(entries_psiex)

1688
43688


e) Write a function to calculate the overlap between two MPS. Recall from class that
there is an inefficient way (first contracting the bra and ket on top and bottom
separately and finally contracting over the j1, . . . jn) and an efficient way (contracting from left to right); implement the efficient one! Check that the overlap is (close to) 1 and calculate the overlap

In [10]:
def overlap(M1,M2):
    #M1 and M2 are the two sets of MPS of the two states
    #first element of each list is the matrix on the left
    N = len(M1)
    #print(len(M1))
    #print(len(M2))
    tensor2 = np.tensordot(M2[0].conj(), M1[0], [[0, 1], [0, 1]]) # [vL*] [i*] vR*, [vL] [i] vR -> vR* vR
    for i in range(1,N-1):
        tensor1 = np.tensordot(tensor2, M1[i], [1,0]) # vR* [vR], [vL] i vR -> vR* i vR
        tensor2 = np.tensordot(M2[i].conj(), tensor1, [[0,1],[0,1]]) # [vL*] [i*] vR*, [vR*] [i] vR -> vR* vR
    tensor1 = np.tensordot(tensor2, M1[N-1], [1,0]) # vR* [vR], [vL] i vR -> vR* i vR
    result = np.tensordot(M2[N-1].conj(), tensor1, [[0,1,2],[0,1,2]]) # [vL*] [i*] [vR*], [vR*] [i] [vR] -> [vR*] [vR]
    return result

In [11]:
print(overlap(psiex, psiex))
print()
print(overlap(psiex, psicompr))

1.0000000000000013

0.9999999999999905


f) Write the state |↑↑ · · · ↑⟩ as an MPS with bond dimension 1. Calculate the overlap
of this state with the ground state (using MPS techniques, i.e. use the function you
wrote in e) )

In [13]:
L = 14
g = 1.5
J = 0.
sx_list = ed.gen_sx_list(L)
sz_list = ed.gen_sz_list(L)
H = ed.gen_hamiltonian(sx_list, sz_list, g, J)
E, vecs = scipy.sparse.linalg.eigsh(H, which='SA')
psinew = vecs[:, 0]
print(psinew)
np.linalg.norm(psinew)

#ground state of Hamiltonian with J = 0 is exactly |↑↑ ·· ↑⟩
# which in comp basis is written as (1,0,0...0)

[ 1.00000000e+00  1.31120353e-16  1.22888407e-16 ...  1.07955461e-17
  1.07100075e-17 -9.42280401e-17]


1.0000000000000007

In [14]:
psinew_MPS = compress(psinew, 14, 1)

In [15]:
print("Overlap of psinew and groundstate: ", overlap(psinew_MPS, psiex))

Overlap of psinew and groundstate:  0.814694334791425


## Ex 7.2

First construct a dimerized spin-1
2 chain of singlets

In [46]:
M1up = np.array([1/np.sqrt(2),0])
M1down = np.array([0, 1/np.sqrt(2)])
M2up = np.array([[0],[1]])
M2down = np.array([[1],[0]])



In [47]:
N = 14 #number of 1-spins
tensor1 = np.array([M1up, M1down])
tensor1 = np.reshape(tensor1, (1,2,2))
tensor2 = np.hstack((M2up, M2down))
tensor2 = np.reshape(tensor2, (2,2,1))
spinchain = []
for i in range(N):
    spinchain.append(tensor1)
    print(np.shape(np.array(spinchain[i])))
    spinchain.append(tensor2)

print(spinchain)
print()
print(spinchain[4])

(1, 2, 2)
(2, 2, 1)
(1, 2, 2)
(2, 2, 1)
(1, 2, 2)
(2, 2, 1)
(1, 2, 2)
(2, 2, 1)
(1, 2, 2)
(2, 2, 1)
(1, 2, 2)
(2, 2, 1)
(1, 2, 2)
(2, 2, 1)
[array([[[0.70710678, 0.        ],
        [0.        , 0.70710678]]]), array([[[0],
        [1]],

       [[1],
        [0]]]), array([[[0.70710678, 0.        ],
        [0.        , 0.70710678]]]), array([[[0],
        [1]],

       [[1],
        [0]]]), array([[[0.70710678, 0.        ],
        [0.        , 0.70710678]]]), array([[[0],
        [1]],

       [[1],
        [0]]]), array([[[0.70710678, 0.        ],
        [0.        , 0.70710678]]]), array([[[0],
        [1]],

       [[1],
        [0]]]), array([[[0.70710678, 0.        ],
        [0.        , 0.70710678]]]), array([[[0],
        [1]],

       [[1],
        [0]]]), array([[[0.70710678, 0.        ],
        [0.        , 0.70710678]]]), array([[[0],
        [1]],

       [[1],
        [0]]]), array([[[0.70710678, 0.        ],
        [0.        , 0.70710678]]]), array([[[0],
       

Use the overlap function between two MPS that you wrote in the previous exercise
to check the norm of this singlet MPS, and to compute the spin-correlation function
$<σ^z_i σ^z_j>$ . You should observe that this gives always 0 for |i − j| > 1

In [48]:
norm_spinchain = overlap(spinchain, spinchain)
print(norm_spinchain)

0.9999999999999978


Credo che col fatto che in questa chain di spin 1/2 singlets, bisogna applicare sigma z ai due nodi adiacenti che rappresentano lo spin-1. Quindi se devo applicare sigma z nella posizione j, siccome il primo nodo è il secondo elemento del primo Spin-1 (j=1), la posizione j corrisponde ai nodi: j e j+1 -> [j-1] e [j]

In [1]:
#credo che col fatto che
"""def sigmazspinchain(j,spinchain):
    sigmaz = np.array([[1,0],[0,-1]])
    result = spinchain
    #print((spinchain[j-1]))
    #print()
    tensor = np.tensordot(result[j-1], sigmaz, ([1,1])) # vR [i] vL, i [i*]-> vR vL i
    tensor = tensor.transpose([0,2,1])
    result[j-1] = tensor
    tensor1 = np.tensordot(result[j], sigmaz, ([1,1])) # vR [i] vL, i [i*]-> vR vL i
    tensor1 = tensor1.transpose([0,2,1])
    result[j] = tensor1
    #print(spinchain[j-1])
    return result"""

def sigmazspinchain(j,spinchain):
    sigmaz = np.array([[1,0],[0,-1]]) 
    result = spinchain.copy() #lists are always passed by reference, arrays are copied (per essere sicuro fai deepcopy)
    #print((spinchain[j-1]))
    #print()
    #print(np.shape(np.array(result[2*(j-1)])))
    #print(result[j+1])
    tensor = np.tensordot(result[j], sigmaz, ([1,1])) # vR [i] vL (1,2,2), i [i*] (2,2)-> vR vL i (1,2,2)
    tensor = tensor.transpose([0,2,1])
    result[j] = tensor
    #print(3,tensor)

    #print(np.shape(np.array(result[2*(j-1)])))
    #print(np.shape(np.array(result[2*j])))
    #tensor1 = np.tensordot(result[2*j], sigmaz, ([1,1])) # vR [i] vL (2,2,1), i [i*] (2,2)-> vR vL i (2,1,2)
    #tensor1 = tensor1.transpose([0,2,1]) #(2,2,1)
    #print(np.shape(np.array(tensor1)))
    #result[2*j] = tensor1

    return result

def sigmazijspinchain(i,j,spinchain): 
    return sigmazspinchain(j,sigmazspinchain(i,spinchain))
    
def correlation_function(i,j,spinchain):
    print("patata",sigmazijspinchain(i,j,spinchain)[2])
    return overlap(spinchain, sigmazijspinchain(i,j,spinchain))

print(correlation_function(2,10,spinchain))

NameError: name 'spinchain' is not defined